In [9]:
# Import the required packages
using JuMP
using CSV
using DataFrames
using PowerModels
using Ipopt
using Juniper

In [22]:
idx = findall(isequal(3), load_busses)
if !isempty(idx)
    print("WHOA")
end
size(Pd_mod)

WHOA

(1, 18)

In [25]:
#Read in data
baseMVA = 10;

bus_data = CSV.read("bus_data.csv", DataFrame) |> Matrix;
(num_busses, var) = size(bus_data);
load_data = CSV.read("load_data.csv", DataFrame) |> Matrix;
(num_loads, var) = size(load_data);
branch_data = CSV.read("branch_data.csv", DataFrame) |> Matrix;
(num_branches, var) = size(branch_data)
G = CSV.read("G_mat.csv", DataFrame) |> Matrix;
B = CSV.read("B_mat.csv", DataFrame)|> Matrix;

#Expand the Pd data such that non load busses are given power demands of 0
Pd = load_data[:,5]*baseMVA
load_busses = load_data[:,2];
Pd_mod = zeros(1, num_busses);
for i = 1:num_busses
    idx = findall(isequal(i), load_busses)
    if !isempty(idx)
        Pd_mod[i] = Pd[idx[1]]
    else
        Pd_mod[i] = 0;
    end
end
Pd = Pd_mod

Qd = load_data[:,4]*baseMVA
Qd_mod = zeros(1, num_busses);
for i = 1:num_busses
    idx = findall(isequal(i), load_busses)
    if !isempty(idx)
        Qd_mod[i] = Qd[idx[1]]
    else
        Qd_mod[i] = 0;
    end
end
Qd = Qd_mod


1×18 Matrix{Float64}:
 0.0  0.12  0.25  0.93  2.26  0.5  0.12  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [27]:
optimizer = Juniper.Optimizer
nl_solver = optimizer_with_attributes(Ipopt.Optimizer, "print_level" => 0)

model = Model(optimizer_with_attributes(optimizer, "nl_solver"=>nl_solver))

@variable(model, V[1:num_busses] >= 0)
@variable(model, theta[1:num_busses] >= 0)
@variable(model, PG[1:num_busses] >= 0)
@variable(model, QG[1:num_busses] >= 0)
@variable(model, y[1:num_busses] >= 0)
@variable(model, x[1:num_busses], Bin)
@variable(model, Ploss[1:num_busses] >= 0)
@variable(model, Qloss[1:num_busses] >= 0)

M = 10000;

@constraint(model, [i in 1:num_busses], y[i] <= M*x[i])
@NLconstraint(model, [i in 1:num_busses], PG[i] - Pd[i] + Ploss[i] == sum((V[i]*V[j])*(G[i,j]*cos(theta[i]-theta[j])+B[i,j]*sin(theta[i]-theta[j])) for j in 1:num_busses))
@NLconstraint(model, [i in 1:num_busses], QG[i] - Qd[i] == sum((V[i]*V[j])*(G[i,j]*sin(theta[i]-theta[j])-B[i,j]*cos(theta[i]-theta[j])) for j in 1:num_busses))


18-element Vector{NonlinearConstraintRef{ScalarShape}}:
 ((QG[1] - 0.0) - ((V[1] * V[1]) * (39.87614338007717 * sin(theta[1] - theta[1]) - -105.02287856174772 * cos(theta[1] - theta[1])) + (V[1] * V[2]) * (-26.35477935668656 * sin(theta[1] - theta[2]) - 73.62216785487384 * cos(theta[1] - theta[2])) + (V[1] * V[3]) * (0.0 * sin(theta[1] - theta[3]) - 0.0 * cos(theta[1] - theta[3])) + (V[1] * V[4]) * (0.0 * sin(theta[1] - theta[4]) - 0.0 * cos(theta[1] - theta[4])) + (V[1] * V[5]) * (0.0 * sin(theta[1] - theta[5]) - 0.0 * cos(theta[1] - theta[5])) + (V[1] * V[6]) * (0.0 * sin(theta[1] - theta[6]) - 0.0 * cos(theta[1] - theta[6])) + (V[1] * V[7]) * (0.0 * sin(theta[1] - theta[7]) - 0.0 * cos(theta[1] - theta[7])) + (V[1] * V[8]) * (0.0 * sin(theta[1] - theta[8]) - 0.0 * cos(theta[1] - theta[8])) + (V[1] * V[9]) * (0.0 * sin(theta[1] - theta[9]) - 0.0 * cos(theta[1] - theta[9])) + (V[1] * V[10]) * (-12.838655948903739 * sin(theta[1] - theta[10]) - 16.624074094662987 * cos(theta[1] - theta[

In [ ]:

# Create a new optimization model using Ipopt as the solver
model = Model(Ipopt.Optimizer)

# Define the variables with bounds
@variable(model, x1 >= 0)  # x1 ≥ 0
@variable(model, x2 >= 0)  # x2 ≥ 0

# Define the objective function
@objective(model, Min, (x1 - 1)^2 + (x2 - 2)^2)

# Define the constraints
@constraint(model, x1^2 + x2^2 <= 1)  # Circle constraint

# Solve the optimization problem
optimize!(model)

# Extract the solution
x1_opt = value(x1)
x2_opt = value(x2)
optimal_value = objective_value(model)

# Display the results
println("Optimal solution:")
println("x1 = $x1_opt")
println("x2 = $x2_opt")
println("Objective value = $optimal_value")